# Notebook to learn Alexander Spannier cochains 

In [14]:
import numpy as np
from scipy import sparse
from scipy.sparse import coo_matrix,diags
from scipy.sparse.linalg import inv
import gudhi as gd
import copy
import random
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import torch
import torch.nn as nn
from itertools import permutations
import math


### Some useful functions 

In [15]:

def generate_diagonal_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        x = np.sort(np.random.uniform(low=-1, high=1, size=num_pts).astype('f'))
        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        x_trans = np.random.randint(-5,5)
        y_trans = np.random.randint(-5,5)

        x_values = list(x+noise_x +x_trans)
        y_values = list(np.sin(x+noise_y)+y_trans)

        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)
        
    return Paths


def generate_antidiagonal_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        x = np.sort(np.random.uniform(low=-1, high=1, size=num_pts).astype('f'))

        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        x_trans = np.random.randint(-5,5)
        y_trans = np.random.randint(-5,5)
        x_values = list(x+noise_x +x_trans)
        y_values = list(-np.sin(x+noise_y)+y_trans)
        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)
        
    return Paths
        



def generate_circular_paths(num_paths=100,eps = 0.2, num_pts = 10):
    
    Paths = []
    for i in range(num_paths): 
        endpoint = np.random.randint(0,num_pts)
        
        sample_angles = list(np.sort(np.random.uniform(0,2*np.pi, num_pts)).astype('f'))
        angles= sample_angles[endpoint:]+ sample_angles[:endpoint]
        angles = np.array(angles)
        
        noise_x= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')
        noise_y= np.random.uniform(low= -eps, high = eps,  size = num_pts).astype('f')

        #x_trans = np.random.randint(-5,5)
        #y_trans = np.random.randint(-5,5)

        r = np.random.uniform(0.5, 2.5)

        x_values = r*np.cos(angles)+noise_x
        y_values = r*np.sin(angles)+noise_y

        path = np.stack((x_values,y_values))
    
        Paths.append(path.T)

    return Paths

In [16]:
def perm_parity(lst):
    ''' Given a permutation of the digits 0..N in order as a list, 
    returns its parity (or sign): +1 for even parity; -1 for odd.
    '''
    parity = 1
    for i in range(0,len(lst)-1):
        if lst[i] != i:
            parity *= -1
            mn = min(range(i,len(lst)), key=lst.__getitem__)
            lst[i],lst[mn] = lst[mn],lst[i]
    return parity   

#permute the entries of a simplex in torch tensor format
def permute_simplex(simplex,perm):
    """ permute the entries of a simplex in torch tensor format"""
    permuted_simplex = torch.zeros(simplex.shape)
    for i in range(simplex.shape[0]):
        permuted_simplex[i] = simplex[perm[i]]
    return permuted_simplex



In [17]:
# initialise a AS cochain as an MLP
# m = dimension of space the complex lives in 
# k = dimension of cochain 

m =2
k = 1
f = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, 1)
)

simp = torch.tensor([(1,0),(0,1)]).float() ## a 1-simplex with node values in R^2
print(simp.shape)
simp = simp.reshape(1,-1)
print(simp.shape)
a = f(simp)
print(a)
print(a.shape)



torch.Size([2, 2])
torch.Size([1, 4])
tensor([[0.3696]], grad_fn=<AddmmBackward0>)
torch.Size([1, 1])


In [18]:
## we can also learn l different cochains 
l=2
F = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, l)
)

simp = torch.tensor([(1,0),(0,1)]).float() ## a 1-simplex with node values in R^2
print(simp.shape)
simp = simp.reshape(1,-1)
print(simp)
print(simp.shape)
a = F(simp)[0]
print(a)
print(a.shape)


torch.Size([2, 2])
tensor([[1., 0., 0., 1.]])
torch.Size([1, 4])
tensor([-0.0964, -0.0475], grad_fn=<SelectBackward0>)
torch.Size([2])


In [19]:
def cochain_eval(cochains,simplex):
    """ Evaluate a set of l k-cochains on a k-simplex """
    
    # check size of simplex is compatible with cochain
    assert simplex.shape == ((k+1),m), "dimension of simplex and cochain does not match"
    
    l = cochains[-1].out_features
    new_simplex = simplex.reshape(1,-1)
    out = torch.zeros(l)
    out = cochains(new_simplex)[0]

    return out 

simp = torch.tensor([(1,0),(0,1)]).float()
cochain_eval(F,simp)

tensor([-0.0964, -0.0475], grad_fn=<SelectBackward0>)

In [20]:
# function to obtain the alternation of a cochain on a simplex

def alternate_cochain(cochain,simplex):
    
    l = cochain[-1].out_features
    s = simplex.shape[0]

    alt = torch.zeros(l)
    perm = permutations(range(simplex.shape[0]))

    for i in list(perm): 
        simplex = permute_simplex(simplex,i)   
        alt += perm_parity(list(i))*cochain_eval(cochain,simplex)
    return alt/math.factorial(s)


simp = torch.tensor([(1,0,),(0,4)]).float() 
print(permute_simplex(simp,[1,0]))

l=2
m = 2
k = 1

F = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, l)
)

a = alternate_cochain(F,simp)
print(a)

tensor([[0., 4.],
        [1., 0.]])
tensor([-0.0150,  0.0556], grad_fn=<DivBackward0>)


In [9]:
def cochain_eval_path(cochains,path):
    """ Evaluate a set of l k-cochains on a simplicial complex
     simplicial complex sc as array of simplices"""
    
    out = torch.zeros(path.shape[0],cochains[-1].out_features)

    for i in range(path.shape[0]-1):
        
    
        simplex = torch.tensor((path[i], path[i+1]))  # convert path[i] to a torch tensor efficiently later
        #print(simplex)
        out[i] = cochain_eval(cochains,simplex)
    return out


### example 

m =2
k = 1
f = nn.Sequential(
    nn.Linear(m*(k+1), 10),
    nn.ReLU(),
    nn.Linear(10, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, 1)
)

p0 = generate_diagonal_paths(num_paths=10,eps = 0.2, num_pts = 5)
path = p0[0]

cochain_eval_path(f,path)

/Users/celiahacker/opt/anaconda3/envs/celia/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  # Remove the CWD from sys.path while we load stuff.


tensor([[-0.0200],
        [-0.0440],
        [-0.0619],
        [-0.0551],
        [ 0.0000]], grad_fn=<CopySlices>)

## Learning part

In [37]:
# generate data

num_paths = 100

p0 = generate_diagonal_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)
p1 = generate_antidiagonal_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)
p2 = generate_circular_paths(num_paths=num_paths,eps = 0.2, num_pts = 10)

# join together p0, p1, p2
paths = p0+p1+p2

# generate labels
labels = np.concatenate((np.zeros(num_paths),np.ones(num_paths),2*np.ones(num_paths))).astype('f')

# perform a one hot encoding of the labels and transform to torch
labels = torch.nn.functional.one_hot(torch.tensor(labels).to(torch.int64))


In [47]:
l=3 # three classes so three outputs 
m = 2 # the paths live in R^2
k = 1 # we deal with one simplices 

F = nn.Sequential(
    nn.Linear(m*(k+1), 300),
    nn.ReLU(),
    nn.Linear(300, 200), ## random number 
    nn.ReLU(),
    nn.Linear(200, 50), ## random number 
    nn.ReLU(),
    nn.Linear(50, 10), 
    nn.ReLU(),
    nn.Linear(10, l)
)


In [48]:
epochs = 100

batch_size = len(paths)

orig_labels = np.concatenate((np.zeros(num_paths),np.ones(num_paths),2*np.ones(num_paths)))

optim = torch.optim.SGD(F.parameters(), lr=1e-4, momentum=0.9)

criterion = nn.CrossEntropyLoss()


for e in range(epochs):

    print(e)

    ## shuffle the data
    idx = np.random.permutation(len(paths))
    paths = [paths[i] for i in idx]
    labels = labels[idx]
    orig_labels = orig_labels[idx]

    correct_pred = 0 

    for i in range(len(paths)):

        p = paths[i]
        y = labels[i]
        
        X = cochain_eval_path(F,p)
        X = torch.sum(X, dim = 0) 
        #print(X)

        sm = torch.nn.functional.softmax(X) 
    
        loss = criterion(sm,y.float())
        loss.backward()

        # get the index of the max log-probability
        pred = sm.argmax(keepdim=True).float()
        #print("predictiton = " ,pred)
        #print("original label = ",orig_labels[i])

        if pred == orig_labels[i]: ## 
          correct_pred += 1
        
        #print("y =", y)
        #print("sm =", sm)

        optim.step()

        optim.zero_grad()

    print(correct_pred/len(paths))



0


/Users/celiahacker/opt/anaconda3/envs/celia/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.3566666666666667
1
0.3466666666666667
2
0.35
3
0.3466666666666667
4
0.34
5
0.3433333333333333
6
0.34
7
0.33666666666666667
8
0.34
9
0.3466666666666667
10
0.3433333333333333
11
0.33
12
0.3333333333333333
13
0.3433333333333333
14
0.3433333333333333
15
0.3333333333333333
16
0.33
17
0.33666666666666667
18
0.32666666666666666
19
0.32666666666666666
20
0.33666666666666667
21
0.32666666666666666
22
0.33666666666666667
23
0.32666666666666666
24
0.3233333333333333
25
0.33
26
0.34
27
0.3233333333333333
28
0.33
29
0.33
30
0.3333333333333333
31
0.3233333333333333
32
0.34
33
0.3333333333333333
34
0.3233333333333333
35
0.3333333333333333
36
0.3333333333333333
37
0.33666666666666667
38
0.3333333333333333
39
0.33
40
0.32666666666666666
41
0.33
42
0.33
43
0.33666666666666667
44
0.33666666666666667
45
0.3333333333333333
46
0.33666666666666667
47
0.33666666666666667
48
0.33666666666666667
49
0.33666666666666667
50
0.34
51
0.3333333333333333
52
0.3566666666666667
53
0.33
54
0.34
55
0.33666666666666667
5

### Graveyard 

In [ ]:
## Evaluate one cochain on a simplex 

def cochain_eval(cochain,simplex):

    """ Evaluate a k-cochains on a k-simplex """
    
    # check size of simplex is compatible with cochain
    assert simplex.shape == ((k+1),m), "dimension of simplex and cochain does not match"
    simplex.reshape(1,-1)
    simplex = simplex.reshape(1,-1)
    out = torch.zeros(1)
    out = cochain(simplex)[0]
    print(out.shape)
    return out 


simp = torch.tensor([(1,0),(0,1)]).float()
res = cochain_eval(f,simp)
print(res.shape)
print(res)


torch.Size([1])
torch.Size([1])
tensor([0.0332], grad_fn=<SelectBackward0>)


In [ ]:
def path2sc(path):

    sc = torch.zeros(len(path)-1, 4)
    for i in range(len(path)):
        sc[i,0] = path[i]
        sc[i,1] = path[i+1]



    return sc